In [1]:
import numpy as np
class AntColony:
    def __init__(self, num_cities, num_ants, alpha=1, beta=2, evaporation_rate=0.5, pheromone_init=0.1):
        self.num_cities = num_cities
        self.num_ants = num_ants
        self.alpha = alpha  # Pheromone influence factor
        self.beta = beta    # Heuristic influence factor
        self.evaporation_rate = evaporation_rate
        self.pheromone_init = pheromone_init
        self.pheromones = np.ones((num_cities, num_cities)) * pheromone_init
        self.distances = np.zeros((num_cities, num_cities))  # Distance matrix
        self.ants = np.zeros((num_ants, num_cities), dtype=int)  # Ants memory
        self.path_costs = np.zeros(num_ants)  # Path cost for each ant
    def add_distances(self, distances):
        self.distances = distances
    def run(self, num_iterations):
        for i in range(num_iterations):
            for ant in range(self.num_ants):
                self.ants[ant] = self.ant_tour(ant)
                self.update_pheromones()
        return self.ants, self.path_costs
    def ant_tour(self, ant):
        ant_path = np.zeros(self.num_cities, dtype=int)
        visited_cities = np.zeros(self.num_cities, dtype=bool)
        current_city = np.random.randint(0, self.num_cities)
        visited_cities[current_city] = True
        ant_path[0] = current_city
        for i in range(1, self.num_cities):
            probabilities = self.get_probabilities(visited_cities, current_city)
            next_city = np.random.choice(np.arange(self.num_cities), p=probabilities)
            ant_path[i] = next_city
            visited_cities[next_city] = True
            self.path_costs[ant] += self.distances[current_city, next_city]
            current_city = next_city
        return ant_path
    def get_probabilities(self, visited_cities, current_city):
        pheromone = np.copy(self.pheromones[current_city])
        pheromone[visited_cities] = 0
        distances_inv = 1 / (self.distances[current_city] + 1e-10)  # Avoid division by zero
        heuristic = distances_inv ** self.beta
        probabilities = (pheromone ** self.alpha) * heuristic
        probabilities /= np.sum(probabilities)
        return probabilities
    def update_pheromones(self):
        self.pheromones *= (1 - self.evaporation_rate)
        for ant in self.ants:
            for i in range(self.num_cities - 1):
                city_from, city_to = ant[i], ant[i + 1]
                self.pheromones[city_from, city_to] += 1 / (self.distances[city_from, city_to] + 1e-10)  # Avoid division by zero
# Example usage
num_cities = int(input("Enter the number of cities: "))
num_ants = int(input("Enter the number of ants: "))
aco = AntColony(num_cities=num_cities, num_ants=num_ants)
# Input distances between cities
distances = np.zeros((num_cities, num_cities))
for i in range(num_cities):
    for j in range(num_cities):
        if i != j:
            distances[i][j] = float(input(f"Enter distance between city {i} and {j}: "))
aco.add_distances(distances)
num_iterations = int(input("Enter the number of iterations: "))
ants, path_costs = aco.run(num_iterations)

print("\nAnts tours:")
for i, ant in enumerate(ants):
    print(f"Ant {i}: {ant}, Path cost: {path_costs[i]}")



Ants tours:
Ant 0: [0 2 1], Path cost: 32.0
Ant 1: [0 2 1], Path cost: 21.0
Ant 2: [1 2 0], Path cost: 38.0


In [ ]:
import random
import numpy as np

# Define the distance matrix for the cities
distance_matrix = [
    [0, 10, 4],
    [9, 0, 6],
    [7, 3, 0]
]

# Define parameters for the ACO algorithm
num_ants = 3
num_iterations = 3
evaporation_rate = 0.5
alpha = 1.0
beta = 2.0

# Initialize pheromone trails
pheromone_matrix = np.ones((len(distance_matrix), len(distance_matrix)))

def get_path_distance(path):
    total_distance = 0
    for i in range(len(path) - 1):
        total_distance += distance_matrix[path[i]][path[i + 1]]
    total_distance += distance_matrix[path[-1]][path[0]]  # Return to the starting city
    return total_distance

def select_next_city(current_city, visited_cities):
    unvisited_cities = [city for city in range(len(distance_matrix)) if city not in visited_cities]
    probabilities = []
    for city in unvisited_cities:
        pheromone = pheromone_matrix[current_city][city]
        distance = distance_matrix[current_city][city]
        probability = (pheromone ** alpha) * ((1.0 / distance) ** beta)
        probabilities.append((city, probability))
    probabilities = sorted(probabilities, key=lambda x: x[1], reverse=True)
    selected_city = probabilities[0][0]
    return selected_city

def update_pheromones(pheromone_matrix, ant_paths, evaporation_rate):
    pheromone_matrix *= (1 - evaporation_rate)  # Evaporate pheromones
    for path_distance, path in ant_paths:
        for i in range(len(path) - 1):
            pheromone_matrix[path[i]][path[i + 1]] += 1.0 / path_distance
        pheromone_matrix[path[-1]][path[0]] += 1.0 / path_distance  # Return to the starting city

def ant_colony_optimization(num_ants, num_iterations):
    global pheromone_matrix
    best_path = None
    best_distance = float('inf')
    for _ in range(num_iterations):
        ant_paths = []
        for ant in range(num_ants):
            current_city = random.randint(0, len(distance_matrix) - 1)
            visited_cities = [current_city]
            while len(visited_cities) < len(distance_matrix):
                next_city = select_next_city(current_city, visited_cities)
                visited_cities.append(next_city)
                current_city = next_city
            path_distance = get_path_distance(visited_cities)
            ant_paths.append((path_distance, visited_cities))
            if path_distance < best_distance:
                best_distance = path_distance
                best_path = visited_cities
        update_pheromones(pheromone_matrix, ant_paths, evaporation_rate)
    return best_path, best_distance

best_path, best_distance = ant_colony_optimization(num_ants, num_iterations)
print("Best path:", best_path)
print("Best distance:", best_distance)